In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import chardet 

In [2]:
#定义爬取函数
def crawl_sina_finance_reports(pages):
    base_url = "https://stock.finance.sina.com.cn/stock/go.php/vReport_List/kind/lastest/index.phtml"
    reports = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    for page in range(1, pages + 1):
        url = f"{base_url}?p={page}"
        response = requests.get(url,headers=headers)#
        # 使用chardet检测编码
        detected_encoding = chardet.detect(response.content)['encoding']
        if detected_encoding:
            #print(detected_encoding)
            response.encoding = detected_encoding
        else:
            response.encoding = 'GB2312'  # 如果chardet无法检测到编码，则默认使用GB2312
        soup = BeautifulSoup(response.content)#, 'html.parser'
 
        # 找到所有报道的列表项
        report_items = soup.find_all('tr')[1:]  # 跳过表头
        for item in report_items:
            columns = item.find_all('td')
            if len(columns) >= 4:
                title = columns[1].text.strip()
                kind = columns[2].text.strip()
                date = columns[3].text.strip()
                organization = columns[4].text.strip()
                analysts = columns[5].text.strip()
                reports.append([title, kind, date, organization, analysts])
 
    return reports
 
# 爬取数据
pages = 300
reports_data = crawl_sina_finance_reports(pages)

In [3]:
# 创建DataFrame
df_reports = pd.DataFrame(reports_data, columns=["标题",'报告类型', "发布日期", "机构","研究员"])
df_reports

,标题,报告类型,发布日期,机构,研究员
0,计算机行业周报：KIMI掀起长文本大战 终端智能进展不断,行业,2024-03-25,上海证券有限责任公司,吴婷婷
1,海容冷链(603187)：国外冷冻展示柜快速增长 毛利率改善明显,公司,2024-03-25,西南证券股份有限公司,龚梦泓
2,建筑材料行业周报：3月二手房成交回升 关注中央预算类资金下达...,行业,2024-03-25,上海证券有限责任公司,方晨
3,银行业跟踪：信贷结构持续优化 降准仍有空间,行业,2024-03-25,东兴证券股份有限公司,林瑾璐/田馨宇
4,水晶光电(002273)：2023年新品导入顺利 关注滤光片创新进展,公司,2024-03-25,中国国际金融股份有限公司,贾顺鹤/黄天擎/彭虎/李澄宁/温晗静
...,...,...,...,...,...
5731,水电研究之大江大河：流域研究框架：以大渡河为例,行业,2024-02-27,广发证券股份有限公司,郭鹏/姜涛
5732,基金周报：基金优选与FOF策略周报,基金,2024-02-27,国泰君安证券股份有限公司,孙雨/王继恒/张思宇/李荣敏/张斌/张孙齐
5733,机器人产业周报：巨头加速入局具身智能,策略,2024-02-27,国泰君安证券股份有限公司,肖群稀/鲍雁辛
5734,亚香股份(301220)：香料香精行业细分龙头 全球化布局迈入新篇章,创业板,2024-02-27,国泰君安证券股份有限公司,钟浩/周志鹏


In [4]:
df_reports.to_csv('C:/Users/lenovo/twitter5214/sina_daily.csv', encoding='utf-8',index=False)